In [9]:
from sklearn.linear_model import Ridge
import numpy as np
from tqdm import tqdm
import pandas as pd

nec_cols = ['product_sales_7', 'product_sales_14', 'product_sales_17', 'product_sales_21', 'product_sales_28',
            'product_sales_35', 'product_sales_42', 'weekday_avg_sales', 'week_trend']

train = sub.train[['date', 'sales', 'unique_id', 'weight'] + nec_cols].copy()
train = train.fillna(0)

beg = '05-01-2024'
end = '05-15-2024'

valid = train.loc[(train['date'] > beg) & (train['date'] <= end)]
train = train.loc[train['date'] <= beg]

ids = valid['unique_id'].unique().tolist()
length = len(nec_cols)

constant = 15050
for constant in [1.07e5, 1.075e5, 1.08e5]:
    score = 0
    for id in tqdm(ids):
        t_id = train.loc[train['unique_id'] == id]
        v_id = valid.loc[valid['unique_id'] == id]
        X = t_id[nec_cols]
        y = t_id['sales']
        alpha = constant * np.var(y) / (length * np.sqrt(len(y)))
        try:
            fitted = Ridge(alpha=alpha).fit(X, y)
            score += np.mean(np.abs(fitted.predict(v_id[nec_cols]) - v_id['sales'])) * v_id['weight'].mean() / 1e3
        except:
            pass
    print(f'{constant} {score:.4f}')

100%|██████████| 3660/3660 [00:30<00:00, 119.00it/s]


107000.0 416.7564


100%|██████████| 3660/3660 [00:32<00:00, 111.61it/s]


107500.0 416.7564


100%|██████████| 3660/3660 [00:32<00:00, 113.73it/s]

108000.0 416.7564


In [11]:
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
from tqdm import tqdm
import gc

nec_cols = ['product_sales_7', 'product_sales_14', 'product_sales_17', 'product_sales_21', 'product_sales_28',
            'product_sales_35', 'product_sales_42', 'weekday_avg_sales', 'week_trend']

train = sub.train[['sales', 'unique_id'] + nec_cols].copy()
train = train.fillna(0)

ids = train['unique_id'].unique().tolist()

length = len(nec_cols)

dict = {}
constant = 107500
for id in tqdm(ids):
    id_df = train.loc[train['unique_id'] == id]
    X = id_df[nec_cols]
    y = id_df['sales']
    alpha = constant * np.var(y) / (len(nec_cols) * np.sqrt(len(y)))
    fitted = Ridge(alpha=alpha).fit(X, y)
    dict[id] = {
        'unique_id': id,
        **{'coef_' + col: coef for col, coef in zip(nec_cols, fitted.coef_)},
        'intercept': fitted.intercept_
    }
df = pd.DataFrame(dict).T
gc.collect()
df['unique_id'] = df['unique_id'].astype(int)
display(df)
df.to_csv('./trend_coefs_7.csv', index=False)

 90%|████████▉ | 4843/5390 [00:32<00:03, 156.74it/s]c:\Users\ttfee\OneDrive\Desktop\Rohlik Sales Forecasting\.venv\Lib\site-packages\sklearn\linear_model\_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
100%|██████████| 5390/5390 [00:35<00:00, 151.26it/s]


,unique_id,coef_product_sales_7,coef_product_sales_14,coef_product_sales_17,coef_product_sales_21,coef_product_sales_28,coef_product_sales_35,coef_product_sales_42,coef_weekday_avg_sales,coef_week_trend,intercept
4845,4845,0.164262,0.083957,0.081896,0.062025,0.046126,0.004576,-0.007078,-5.229834e-02,0.133155,15.001966
4829,4829,0.132905,0.165564,0.104163,0.095168,0.066225,0.088753,0.012915,4.866054e-02,0.177673,-2.382919
1755,1755,0.178883,0.047953,0.002860,0.128338,0.068030,0.045298,0.097922,2.195299e-02,0.126534,41.684422
2915,2915,0.212732,0.034569,0.018500,-0.002189,0.029803,0.033475,0.019891,2.525206e-01,0.091134,-8.668981
4484,4484,0.031434,-0.013838,-0.000861,-0.001499,-0.006630,-0.006698,-0.001971,4.545601e-04,0.018669,341.595203
...,...,...,...,...,...,...,...,...,...,...,...
4497,4497,0.000329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.314084e-04,0.000577,59.723297
1772,1772,0.001383,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.197687e-04,0.000649,81.659772
3290,3290,0.002248,0.001591,0.000372,0.000000,0.000000,0.000000,0.000000,-6.189359e-06,0.003584,105.631148
1404,1404,0.002870,0.002368,0.004118,0.000367,0.000000,0.000000,0.000000,8.243043e-04,0.000221,25.478519
